In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist

In [2]:
# Load Fashion MNIST data
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [3]:
# Reshape data to include the channel dimension (28, 28, 1)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [4]:
# Convert grayscale to RGB by repeating the single channel three times
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)

In [5]:
# Normalize the data to [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

In [6]:
# One-hot encode the labels
y_train = to_categorical(y_train, 10)  # Convert to one-hot encoding
y_test = to_categorical(y_test, 10)    # Convert to one-hot encoding


In [7]:
# Create a preprocessing function to resize and convert images to RGB on the fly
def preprocess(image, label):
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.resize(image, (224, 224))  # Resize on the fly
    return image, label


In [8]:
# Convert dataset to tf.data.Dataset for better memory management
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [9]:
# Apply preprocessing in batches
train_ds = train_ds.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)


In [10]:
# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [11]:
# Load the pre-trained ResNet50 model without the top (the fully connected layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [12]:
# Freeze the base model to prevent its weights from being updated during training
base_model.trainable = False

In [13]:
# Add custom layers on top of the base model for Fashion MNIST
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Use global average pooling instead of flattening
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(10, activation='softmax')(x)  # 10 classes in Fashion MNIST


In [14]:
# Create the model
model = Model(inputs=base_model.input, outputs=outputs)

In [15]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Train the model with the dataset pipeline
model.fit(train_ds, validation_data=test_ds, epochs=10)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 213s 104ms/step - accuracy: 0.3803 - loss: 1.6750 - val_accuracy: 0.6429 - val_loss: 0.9861
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 253s 103ms/step - accuracy: 0.6106 - loss: 1.0535 - val_accuracy: 0.6734 - val_loss: 0.8858
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 202s 103ms/step - accuracy: 0.6426 - loss: 0.9658 - val_accuracy: 0.7137 - val_loss: 0.7950
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 202s 103ms/step - accuracy: 0.6597 - loss: 0.9154 - val_accuracy: 0.7189 - val_loss: 0.7836
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 202s 103ms/step - accuracy: 0.6697 - loss: 0.8860 - val_accuracy: 0.7299 - val_loss: 0.7486
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 185s 94ms/step - accuracy: 0.6816 - loss: 0.8554 - val_accuracy: 0.7301 - val_loss: 0.7379
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 201s 94ms/step - accuracy: 0.6842 - loss: 0.8388 - val_accuracy: 0.7313 - val_loss: 0.7191
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 176s 94ms/step - accur

In [17]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 25s 81ms/step - accuracy: 0.7322 - loss: 0.7229
Test Accuracy: 0.7314000129699707
